**Teoría**

El objetivo es calcular valores esperados termodinámicos para el modelo de Ising usando el método de Monte Carlo.

**Distribución de Boltzmann**

Según la mecánica estadística, la estadística de configuraciones de un sistema en equilibirio termodinámico a temperatura $T=1/(k_B\beta)\in [0,\infty)$, viene descripta por la distribución de Boltzmann, en donde

$$ p_s := Z^{-1}e^{-\beta E_s} $$

es la probabilidad de que el sistema se encuentre en la configuración $s$, $E_s$ denota la energía del sistema en la configuración $s$ y

$$ Z := \sum_s e^{-\beta E_s} $$ 

denota la función partición del sistema.

El valor esperado termodinámico de una observable configuracional $O_s$ viene dado por

$$ \langle O \rangle := \sum_s p_sO_s $$

**Modelo de Ising**

En el modelo de Ising de $N$ sitios, el estado configuracional $s$ vienen descripto por $N$ variables de espin clasicos $s_i\in \{-1,1\}$ para $i=1,...,N$, y por una energía configuracional

$$ E_s = -\sum_{ij} J_{ij}s_is_j - \sum_i h_is_i $$

donde $s=(s_1,...,s_N)$, $J_{ij}\in \mathbb{R}$ es una constante de interacción entre los espines $i$ y $j$ que satisface $J_{ii}=0$ y $J_{ij}=J_{ji}$, y $h_i$ es el campo local asociado al spin $i$.

Notar, si $J_{ij}>0$, luego los espines $i$ y $j$ reducen la energía si están alineados, i.e. si $s_i=s_j$, e incrementan la energía en caso contrario, i.e. si $s_i=-s_j$.

El parámetro de orden que nos interesa calcular en el modelo de Ising es

$$ m_s = \frac{1}{N}\sum_i s_i $$

con valor termodinámico

$$ m = \langle m_s \rangle = \sum_s p_sm_s $$

**Algoritmo de Metropolis**

Podemos muestrear configuraciones $s$ del modelo de Ising utilizando el algoritmo de Metropolis.
El mismo consiste en

1. Elegir una configuración inicial, típicamente, una elegida al azar.

2. Elegir un sitio $k$ al azar.

3. Calcular el cambio de energia $\Delta E$ que surge si el espin del sitio $k$ se flipea (i.e. se da vuelta).

4. Generar un número aleatorio $r\in [0,1]$ con distribución uniforme.

5. Si $r<e^{-\beta \Delta E}$, aceptar el espín flip, y en caso contrario, rechazarlo.

6. Volver al paso 2 y repetir.

Entre los pasos 2. y 6. inclusives, existe un loop del algoritmo. $N$ iteraciones de este loop se considera un paso de Monte Carlo, ya que, en promedio, todos los sitios son considerados alguna vez.

Luego de cierto período de termalización (alrededor de $N^2$ pasos de Monte Carlo), este algoritmo comenzará a generar, en sucesivos pasos de Monte Carlo, una secuencia $s^1,s^2,...$ de muestras de configuraciones que son consistentes con la ecuación de Boltzmann.

Notar que si flipeamos el espín $k$-ésimo, tenemos $s_i^{t+1}=s_i^{t}$ para todo $i\neq k$ y $s_k^{t+1}=-s_k^{t}$.
Luego, el cambio de energía resultante es

\begin{eqnarray}
\Delta E = E^{t+1}-E^t 
&=& 
-\sum_{ij} J_{ij}s_i^{t+1}s_j^{t+1} -\sum_i h_is_i^{t+1}
+\sum_{ij} J_{ij}s_i^ts_j^t +\sum_i h_is_i^t
\\
&=& 
-\sum_{i,j\neq k} J_{ij}(s_i^{t+1}s_j^{t+1}-s_i^ts_j^t) 
-\sum_{i\neq k} J_{ik}(s_i^{t+1}s_k^{t+1}-s_i^ts_k^t) 
-\sum_{j\neq k} J_{kj}(s_k^{t+1}s_j^{t+1}-s_k^ts_j^t) 
-J_{kk}(s_k^{t+1}s_k^{t+1}-s_k^ts_k^t) 
\\
&&
-\sum_{i\neq k} h_i(s_i^{t+1}-s_i^t)
-h_k(s_k^{t+1}-s_k^t)
\\
&=& 
2s_k^{t}\bigg(h_k+2\sum_jJ_{kj}s_j^t\bigg)
\end{eqnarray}

Esto es muy útil para calcular $\Delta E$ sin necesidad de flipear el espín.

In [ ]:
using Statistics
using Plots

In [ ]:
mutable struct Model
    J::Array{Float64,2}
    h::Array{Float64,1}
    s::Array{Float64,1}
    beta::Float64
    function Model(N::Int64,beta::Float64)
        new(zeros(N,N),zeros(N),ones(N),1.0)
    end    
end

In [ ]:
model_size(model::Model) = size(model.h)[1]

In [ ]:
[1,2,3].*[4,5,6]

In [ ]:
function metropolis_step!(model::Model,k::Int64)
    DE = 2.0*model.s[k]*(model.h[k]+2.0*sum(model.J[k,:].*model.s[:]))
    if rand()<exp(-model.beta*DE)
        model.s[k] *= -1.0
    end
end

In [ ]:
function monte_carlo_step!(model::Model)
    N = model_size(model)
    for i in 1:N
        k = rand(1:N)
        metropolis_step!(model,k)
    end
end

In [ ]:
function init_model!(model::Model)
    N = model_size(model)
    for i in 1:N
        for j in i+1:N
            model.J[i,j] = 1.0/N
            model.J[j,i] = 1.0/N
        end
    end
end

In [ ]:
function random_state!(model)
    N = model_size(model)
    for i in 1:N
        model.s[i] = rand(0:1)
    end
    model.s .*= 2.0
    model.s .-= 1.0
end

In [ ]:
function order_parameter(model::Model)
   sum(model.s)/model_size(model)
end

In [ ]:
model = Model(128,1.0)

In [ ]:
init_model!(model)
random_state!(model)
model.s

In [ ]:
order_parameter(model)

In [ ]:
delta_T = 0.05
T_max = 2.0
list_m = []
for T in delta_T:delta_T:T_max
    model.beta = 1.0/T
    N = model_size(model)
    # Termalizamos
    random_state!(model)    
    for mc_step in 1:10*N*N
        monte_carlo_step!(model)
    end
    # Medimos configuraciones
    mc_step_max = N*N
    list_abs_m = []
    for mc_step in 1:mc_step_max
        monte_carlo_step!(model)
        m = order_parameter(model)
        #println("beta=",beta," mc_step=",mc_step," m=",m," model.s=",model.s)
        push!(list_abs_m,abs(m))
    end
    println("T=",T," <|m|>=",mean(list_abs_m)," std_{|m|}=",std(list_abs_m))
    push!(list_m,mean(list_abs_m))
end

In [ ]:
plot(delta_T:delta_T:T_max,list_m,xlabel="T",ylabel="|m|")